# 📧 Tutorial 3: Extrator de Email Inteligente

## 🎯 Objetivos de Aprendizado
Ao final deste tutorial, você será capaz de:
- **Criar prompts estruturados** para extrair dados específicos
- **Usar Output Parsers** para formatar respostas da IA
- **Implementar validação de dados** com Pydantic
- **Processar emails** de forma inteligente e automatizada
- **Construir sistemas de categorização** automática

## 📚 Conceitos que vamos dominar

### 1. **PromptTemplate vs ChatPromptTemplate**
- **PromptTemplate**: Para modelos de texto simples (GPT-3.5, etc.)
- **ChatPromptTemplate**: Para modelos conversacionais (ChatGPT, etc.)
- **Diferença**: ChatPromptTemplate inclui roles (system, human, assistant)

### 2. **Output Parsers**
- **StrOutputParser**: Converte resposta em string simples
- **PydanticOutputParser**: Converte resposta em objeto estruturado
- **JSONOutputParser**: Converte resposta em JSON válido

### 3. **Pydantic para Validação**
- **BaseModel**: Classe base para modelos de dados
- **Field**: Define campos com validação e descrição
- **EmailStr**: Validação específica para emails
- **Validação automática**: Garante integridade dos dados

### 4. **Regex para Processamento de Texto**
- **Padrões de email**: `r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'`
- **Extração de dados**: Encontrar informações específicas no texto
- **Validação local**: Verificar formatos antes de enviar para IA

## 🚀 Por que este tutorial é importante?

### **Aplicações Reais:**
- **Sistema de triagem de emails** para empresas
- **Análise de sentimento** em comunicações
- **Categorização automática** de mensagens
- **Dashboard de monitoramento** de comunicações

### **Habilidades Desenvolvidas:**
- **Estruturação de dados** com modelos robustos
- **Validação de entrada** para evitar erros
- **Processamento em lote** para eficiência
- **Exportação de dados** para análise posterior

In [4]:
# Imports do LangChain
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI

# Imports do Pydantic para validação de dados
from pydantic import BaseModel, Field, EmailStr
from typing import List, Optional

# Imports para processamento de dados
from datetime import datetime
import re  # Para processamento de texto com padrões
import json  # Para exportação de dados

# Imports para configuração
from dotenv import load_dotenv

# Carrega variáveis de ambiente (API keys)
load_dotenv()

True

### 🏗️ DEFINIÇÃO DOS MODELOS DE DADOS

Explicação: Por que usar Pydantic?
- Validação automática de tipos
- Documentação automática dos campos
- Serialização/deserialização fácil
- Integração perfeita com LangChain

In [6]:
class InformacoesEmail(BaseModel):
    """
    Modelo para informações extraídas de um email
    
    Este modelo define a estrutura dos dados que queremos extrair
    de cada email, com validação automática de tipos.
    """
    # Campo obrigatório com validação de email
    remetente: str = Field(
        description="Nome completo do remetente do email",
        min_length=2,
        max_length=100
    )
    
    # Validação específica para email
    email_remetente: EmailStr = Field(
        description="Endereço de email do remetente (formato válido)"
    )
    
    # Assunto do email
    assunto: str = Field(
        description="Assunto/título do email",
        min_length=5,
        max_length=200
    )
    
    # Data de envio
    data_envio: str = Field(
        description="Data e hora de envio do email (formato: DD/MM/AAAA HH:MM)"
    )
    
    # Prioridade com valores específicos
    prioridade: str = Field(
        description="Nível de prioridade do email",
        pattern="^(alta|média|baixa|urgente)$"
    )
    
    # Categoria do email
    categoria: str = Field(
        description="Categoria do email (trabalho, pessoal, spam, newsletter, etc.)",
        min_length=3,
        max_length=50
    )
    
    # Lista de palavras-chave importantes
    palavras_chave: List[str] = Field(
        description="Lista de palavras-chave importantes encontradas no email",
        min_length=1,
        max_length=10
    )
    
    # Resumo do conteúdo
    resumo: str = Field(
        description="Resumo conciso do conteúdo do email (máximo 200 caracteres)",
        max_length=200
    )
    
    # Se requer ação
    acao_necessaria: bool = Field(
        description="Indica se o email requer alguma ação do destinatário"
    )
    
    # Nível de urgência (1-5)
    urgencia: int = Field(
        description="Nível de urgência do email (1=baixa, 5=crítica)",
        ge=1,
        le=5
    )

class ListaEmails(BaseModel):
    """
    Modelo para lista de emails processados
    
    Este modelo agrupa múltiplos emails e inclui estatísticas
    para análise em lote.
    """
    # Lista de emails processados
    emails: List[InformacoesEmail] = Field(
        description="Lista de emails processados e categorizados"
    )
    
    # Total de emails
    total_emails: int = Field(
        description="Número total de emails processados"
    )
    
    # Contagem por categoria
    categorias: dict = Field(
        description="Dicionário com contagem de emails por categoria"
    )

print("✅ Modelos de dados definidos!")
print("📋 Validação automática configurada!")
print("🔍 Campos documentados com descrições!")

✅ Modelos de dados definidos!
📋 Validação automática configurada!
🔍 Campos documentados com descrições!


### 🔧 CONFIGURAÇÃO DOS OUTPUT PARSERS

Explicação: O que são Output Parsers?
- Convertem respostas de texto da IA em dados estruturados
- Garantem que os dados estejam no formato correto
- Facilitam a integração com sistemas de banco de dados
- Reduzem erros de parsing manual

In [7]:
def configurar_parser():
    """
    Configura o parser para extrair dados de um email individual
    
    Este parser converte a resposta da IA em um objeto InformacoesEmail
    com validação automática de todos os campos.
    """
    parser = PydanticOutputParser(pydantic_object=InformacoesEmail)
    return parser

def configurar_parser_lista():
    """
    Configura o parser para processar múltiplos emails
    
    Este parser converte a resposta da IA em um objeto ListaEmails
    que contém todos os emails processados e estatísticas.
    """
    parser = PydanticOutputParser(pydantic_object=ListaEmails)
    return parser

# Teste do parser
parser = configurar_parser()
print("✅ Output Parser configurado!")
print("📋 Formato esperado pela IA:")
print(parser.get_format_instructions())
print("\n💡 A IA receberá estas instruções para formatar sua resposta corretamente!")

✅ Output Parser configurado!
📋 Formato esperado pela IA:
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Modelo para informações extraídas de um email\n\nEste modelo define a estrutura dos dados que queremos extrair\nde cada email, com validação automática de tipos.", "properties": {"remetente": {"description": "Nome completo do remetente do email", "maxLength": 100, "minLength": 2, "title": "Remetente", "type": "string"}, "email_remetente": {"description": "Endereço de email do remetente (formato válido)", "format": "email", "title": "Email Remetente", "type": "string"}, "

### 📋 CRIAÇÃO DE PROMPTS ESTRUTURADOS

Explicação: Por que usar prompts estruturados?
- Garantem consistência nas respostas da IA
- Reduzem ambiguidade e erros de interpretação
- Facilitam a manutenção e atualização
- Permitem reutilização em diferentes contextos

In [8]:

def criar_prompt_extrator():
    """
    Cria prompt para extrair informações de um email individual
    
    Este prompt instrui a IA sobre:
    - O que extrair (campos específicos)
    - Como formatar (JSON válido)
    - Qual o contexto (análise de email)
    """
    template = """
    Você é um especialista em análise de emails.
    
    Sua tarefa é extrair informações específicas do email fornecido e retornar
    os dados no formato JSON exato especificado abaixo.
    
    {format_instructions}
    
    IMPORTANTE:
    - Analise cuidadosamente o conteúdo do email
    - Identifique a prioridade baseada no tom e urgência
    - Categorize o email baseado no assunto e conteúdo
    - Extraia palavras-chave relevantes
    - Determine se há necessidade de ação
    - Avalie o nível de urgência de 1 a 5
    
    Email para análise:
    {email}
    
    Responda APENAS com o JSON válido, sem explicações adicionais.
    """
    
    prompt = PromptTemplate(
        template=template,
        input_variables=["email"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    
    return prompt

def criar_prompt_categorizador():
    """
    Cria prompt para categorizar múltiplos emails
    
    Este prompt instrui a IA sobre:
    - Como processar múltiplos emails
    - Como categorizar cada um
    - Como gerar estatísticas
    """
    template = """
    Você é um especialista em organização e categorização de emails.
    
    Sua tarefa é analisar a lista de emails fornecida e:
    1. Extrair informações de cada email
    2. Categorizar cada email adequadamente
    3. Gerar estatísticas de categorização
    
    {format_instructions}
    
    DIRETRIZES DE CATEGORIZAÇÃO:
    - trabalho: Emails relacionados ao trabalho/profissional
    - pessoal: Emails pessoais/familiares
    - spam: Emails suspeitos ou indesejados
    - newsletter: Boletins informativos
    - financeiro: Emails bancários/financeiros
    - urgente: Emails que requerem atenção imediata
    
    Lista de emails:
    {emails}
    
    Responda APENAS com o JSON válido, sem explicações adicionais.
    """
    
    prompt = PromptTemplate(
        template=template,
        input_variables=["emails"],
        partial_variables={"format_instructions": configurar_parser_lista().get_format_instructions()}
    )
    
    return prompt

print("✅ Prompts estruturados criados!")
print("📋 Instruções detalhadas para a IA configuradas!")
print("🎯 Contexto e diretrizes definidos!")

✅ Prompts estruturados criados!
📋 Instruções detalhadas para a IA configuradas!
🎯 Contexto e diretrizes definidos!


### 🤖 CONFIGURAÇÃO DO MODELO E CHAIN

Explicação: O que é uma Chain no LangChain?
- Uma sequência de operações conectadas
- Cada operação passa seu resultado para a próxima
- Facilita a construção de pipelines complexos
- Permite reutilização e modularidade

In [9]:

# Modelo de IA (configuração otimizada para extração)
chat_model = ChatOpenAI(
    temperature=0.0,  # Baixa criatividade para consistência
    model_name="gpt-3.5-turbo"  # Modelo rápido e eficiente
)
    

def configurar_extrator():
    """
    Configura o extrator de emails individual
    
    Chain: Prompt → Modelo → Parser
    - Prompt: Instrui a IA sobre o que fazer
    - Modelo: Processa o texto e gera resposta
    - Parser: Converte resposta em dados estruturados
    """
    # Parser para dados estruturados
    parser = configurar_parser()
    
    # Prompt estruturado
    prompt = criar_prompt_extrator()
    
    # Chain: conecta todas as operações
    chain = prompt | chat_model | parser
    
    return chain

def configurar_categorizador():
    """
    Configura o categorizador de múltiplos emails
    
    Chain: Prompt → Modelo → Parser
    - Prompt: Instrui sobre categorização em lote
    - Modelo: Processa múltiplos emails
    - Parser: Converte em lista estruturada
    """
    # Parser para lista de emails
    parser = configurar_parser_lista()
    
    # Prompt para categorização
    prompt = criar_prompt_categorizador()
    
    # Chain: conecta todas as operações
    chain = prompt | chat_model | parser
    
    return chain

print("✅ Extrator e categorizador configurados!")
print("🔗 Chains criadas: Prompt → Modelo → Parser")
print("⚙️ Configuração otimizada para consistência!")

✅ Extrator e categorizador configurados!
🔗 Chains criadas: Prompt → Modelo → Parser
⚙️ Configuração otimizada para consistência!


### 📧 FUNÇÕES DE PROCESSAMENTO DE EMAILS

Explicação: Por que separar em funções?
- Código mais limpo e organizado
- Facilita testes e manutenção
- Permite reutilização
- Melhora a legibilidade

In [10]:

def extrair_informacoes_email(email_texto, extrator):
    """
    Extrai informações de um email usando IA
    
    Esta função é o coração do sistema de extração:
    1. Recebe o texto do email
    2. Envia para a IA processar
    3. Retorna dados estruturados e validados
    
    Args:
        email_texto (str): Texto completo do email
        extrator: Chain configurado para extração
    
    Returns:
        InformacoesEmail: Dados extraídos e validados
        None: Se houver erro na extração
    """
    try:
        # Invoca a chain para processar o email
        resultado = extrator.invoke({"email": email_texto})
        
        # Validação adicional (Pydantic já valida automaticamente)
        if resultado and hasattr(resultado, 'remetente'):
            print(f"✅ Email processado: {resultado.assunto}")
            return resultado
        else:
            print("⚠️ Dados incompletos retornados")
            return None
            
    except Exception as e:
        print(f"❌ Erro ao extrair informações: {e}")
        return None

def processar_multiplos_emails(emails_texto, categorizador):
    """
    Processa múltiplos emails de uma vez (processamento em lote)
    
    Esta função é otimizada para processar muitos emails:
    1. Formata a lista de emails
    2. Envia tudo para a IA de uma vez
    3. Retorna dados estruturados de todos os emails
    
    Args:
        emails_texto (List[str]): Lista de textos de emails
        categorizador: Chain configurado para categorização
    
    Returns:
        ListaEmails: Dados processados de todos os emails
        None: Se houver erro no processamento
    """
    try:
        # Formata a lista de emails para a IA
        emails_formatados = "\n\n".join([
            f"Email {i+1}:\n{email}" 
            for i, email in enumerate(emails_texto)
        ])
        
        # Processa todos os emails de uma vez
        resultado = categorizador.invoke({"emails": emails_formatados})
        
        if resultado and hasattr(resultado, 'emails'):
            print(f"✅ Processados {resultado.total_emails} emails")
            return resultado
        else:
            print("⚠️ Dados incompletos retornados")
            return None
            
    except Exception as e:
        print(f"❌ Erro ao processar emails: {e}")
        return None

def validar_email_regex(email):
    """
    Valida formato de email usando regex (validação local)
    
    Por que validar localmente?
    - Reduz chamadas desnecessárias para a IA
    - Validação mais rápida
    - Reduz custos de API
    - Melhora a experiência do usuário
    
    Args:
        email (str): Endereço de email para validar
    
    Returns:
        bool: True se válido, False caso contrário
    """
    # Padrão regex para validação de email
    padrao = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(padrao, email) is not None

def extrair_emails_regex(texto):
    """
    Extrai emails de um texto usando regex
    
    Útil para:
    - Encontrar emails em textos longos
    - Validar presença de emails
    - Extrair múltiplos emails de uma vez
    
    Args:
        texto (str): Texto para extrair emails
    
    Returns:
        List[str]: Lista de emails encontrados
    """
    # Padrão regex para encontrar emails
    padrao = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    emails = re.findall(padrao, texto)
    return emails

print("✅ Funções de processamento criadas!")
print("🔍 Validação local implementada!")
print("⚡ Processamento em lote otimizado!")

✅ Funções de processamento criadas!
🔍 Validação local implementada!
⚡ Processamento em lote otimizado!


### 🧪 TESTE COMPLETO DO EXTRATOR DE EMAIL

In [ ]:
def teste_completo_extrator():
    """
    Testa todas as funcionalidades do extrator de email
    """
    print("🚀 TESTE COMPLETO DO EXTRATOR DE EMAIL")
    print("=" * 60)
    
    # 1. Configura o extrator
    print("1️⃣ Configurando extrator...")
    extrator = configurar_extrator()
    categorizador = configurar_categorizador()
    print("✅ Extrator configurado!")
    
    # 2. Emails de teste variados
    print("\n2️⃣ Preparando emails de teste...")
    emails_teste = [
        # Email de trabalho urgente
        """
        De: João Silva <joao.silva@empresa.com>
        Para: maria.santos@empresa.com
        Assunto: URGENTE: Reunião cancelada - Projeto X
        Data: 15/03/2024 14:30
        
        Olá Maria,
        
        A reunião de hoje às 16h foi CANCELADA devido a problemas técnicos.
        Preciso reagendar para amanhã às 10h. É URGENTE pois o prazo está apertado.
        
        Por favor, confirme sua disponibilidade.
        
        Atenciosamente,
        João Silva
        Gerente de Projetos
        """,
        
        # Email pessoal
        """
        De: Ana Costa <ana.costa@gmail.com>
        Para: lucas.biason@outlook.com
        Assunto: Convite para aniversário
        Data: 16/03/2024 19:45
        
        Oi Lucas!
        
        Como vai? Quero te convidar para meu aniversário no sábado!
        Será na minha casa às 20h. Vai ter bolo, música e muita diversão.
        
        Confirma se vai conseguir vir? Preciso saber para organizar a comida.
        
        Beijos,
        Ana
        """,
        
        # Email spam
        """
        De: Ganhe Dinheiro <spam@falso.com>
        Para: usuario@email.com
        Assunto: GANHE R$ 10.000 POR MÊS TRABALHANDO DE CASA!
        Data: 17/03/2024 08:00
        
        ATENÇÃO! OPORTUNIDADE ÚNICA!
        
        Descubra o segredo que os bancos não querem que você saiba!
        Ganhe R$ 10.000 por mês sem sair de casa!
        
        Clique AQUI agora: [LINK SUSPEITO]
        
        Oferta válida apenas por 24 horas!
        """,
        
        # Newsletter
        """
        De: Newsletter Tech <newsletter@techblog.com>
        Para: assinante@email.com
        Assunto: Newsletter Semanal - IA e Machine Learning
        Data: 18/03/2024 09:00
        
        Olá Assinante,
        
        Esta semana trouxemos as principais novidades em IA:
        
        �� OpenAI lança GPT-5
        🤖 Google anuncia novo modelo de linguagem
        📊 Estudo mostra crescimento de 300% no uso de IA
        
        Leia mais em nosso blog: [LINK]
        
        Equipe TechBlog
        """,
        
        # Email financeiro
        """
        De: Banco Digital <noreply@bancodigital.com>
        Para: cliente@email.com
        Assunto: Extrato mensal disponível
        Data: 19/03/2024 07:30
        
        Prezado Cliente,
        
        Seu extrato de março/2024 está disponível para download.
        
        Resumo da conta:
        - Saldo atual: R$ 2.450,00
        - Movimentações: 15 transações
        - Limite disponível: R$ 5.000,00
        
        Acesse sua conta para visualizar o extrato completo.
        
        Banco Digital
        """
    ]
    
    print(f"✅ {len(emails_teste)} emails de teste preparados!")
    
    # 3. Testa extração individual
    print("\n3️⃣ Testando extração individual...")
    resultados_individuais = []
    
    for i, email in enumerate(emails_teste):
        print(f"\n📧 Processando Email {i+1}...")
        resultado = extrair_informacoes_email(email, extrator)
        
        if resultado:
            resultados_individuais.append(resultado)
            print(f"✅ Extraído: {resultado.assunto}")
            print(f"   �� De: {resultado.remetente} ({resultado.email_remetente})")
            print(f"   🏷️ Categoria: {resultado.categoria}")
            print(f"   ⚡ Prioridade: {resultado.prioridade}")
            print(f"   🚨 Urgência: {resultado.urgencia}/5")
            print(f"   🎯 Ação necessária: {'Sim' if resultado.acao_necessaria else 'Não'}")
        else:
            print(f"❌ Falha na extração do Email {i+1}")
    
    # 4. Testa processamento em lote
    print(f"\n4️⃣ Testando processamento em lote...")
    resultado_lote = processar_multiplos_emails(emails_teste, categorizador)
    
    if resultado_lote:
        print(f"✅ Processados {resultado_lote.total_emails} emails em lote")
        print(f"📊 Categorias encontradas: {list(resultado_lote.categorias.keys())}")
        
        # Mostra estatísticas
        print("\n📈 ESTATÍSTICAS:")
        for categoria, count in resultado_lote.categorias.items():
            print(f"   {categoria}: {count} emails")
    
    # 5. Testa validações
    print(f"\n5️⃣ Testando validações...")
    emails_validos = [
        "joao@empresa.com",
        "maria.santos@gmail.com",
        "lucas+teste@outlook.com"
    ]
    
    emails_invalidos = [
        "email-invalido",
        "@dominio.com",
        "usuario@",
        "teste@dominio"
    ]
    
    print("✅ Emails válidos:")
    for email in emails_validos:
        print(f"   {email}: {validar_email_regex(email)}")
    
    print("❌ Emails inválidos:")
    for email in emails_invalidos:
        print(f"   {email}: {validar_email_regex(email)}")
    
    # 6. Testa extração com regex
    print(f"\n6️⃣ Testando extração com regex...")
    texto_com_emails = """
    Contato: joao@empresa.com
    Suporte: suporte@tech.com
    Vendas: vendas@loja.com.br
    """
    
    emails_encontrados = extrair_emails_regex(texto_com_emails)
    print(f"�� Emails encontrados: {emails_encontrados}")
    
    # 7. Resumo final
    print(f"\n7️⃣ RESUMO FINAL:")
    print(f"✅ Emails processados individualmente: {len(resultados_individuais)}")
    print(f"✅ Emails processados em lote: {resultado_lote.total_emails if resultado_lote else 0}")
    print(f"✅ Validações testadas: {len(emails_validos + emails_invalidos)}")
    print(f"✅ Extração regex testada: {len(emails_encontrados)} emails encontrados")
    
    print(f"\n�� TESTE COMPLETO FINALIZADO COM SUCESSO!")
    
    return resultados_individuais, resultado_lote

# Executa o teste completo
resultados_ind, resultados_lote = teste_completo_extrator()

🚀 TESTE COMPLETO DO EXTRATOR DE EMAIL
1️⃣ Configurando extrator...
✅ Extrator configurado!

2️⃣ Preparando emails de teste...
✅ 5 emails de teste preparados!

3️⃣ Testando extração individual...

📧 Processando Email 1...
✅ Email processado: URGENTE: Reunião cancelada - Projeto X
✅ Extraído: URGENTE: Reunião cancelada - Projeto X
   �� De: João Silva (joao.silva@empresa.com)
   🏷️ Categoria: trabalho
   ⚡ Prioridade: urgente
   🚨 Urgência: 5/5
   🎯 Ação necessária: Sim

📧 Processando Email 2...
✅ Email processado: Convite para aniversário
✅ Extraído: Convite para aniversário
   �� De: Ana Costa (ana.costa@gmail.com)
   🏷️ Categoria: pessoal
   ⚡ Prioridade: baixa
   🚨 Urgência: 3/5
   🎯 Ação necessária: Sim

📧 Processando Email 3...
✅ Email processado: GANHE R$ 10.000 POR MÊS TRABALHANDO DE CASA!
✅ Extraído: GANHE R$ 10.000 POR MÊS TRABALHANDO DE CASA!
   �� De: Ganhe Dinheiro (spam@falso.com)
   🏷️ Categoria: spam
   ⚡ Prioridade: alta
   🚨 Urgência: 5/5
   🎯 Ação necessária: Sim

📧 Pr

## 🎯 Desafios para Praticar

### **Desafio 1: Sistema de Filtros Avançados** 🔍
**Objetivo:** Implementar um sistema de filtros que permita buscar emails por múltiplos critérios simultaneamente.

**O que você deve fazer:**
- Criar uma função que filtre emails por categoria (trabalho, pessoal, spam, etc.)
- Implementar filtros por prioridade (alta, média, baixa, urgente)
- Adicionar filtro por nível de urgência (1-5)
- Criar filtro por emails que requerem ação
- Implementar busca por palavras-chave específicas
- Permitir combinação de múltiplos filtros

**Exemplo de uso:**
```python
emails_urgentes = filtrar_emails_por_criterios(emails, {
    'urgencia_minima': 4,
    'acao_necessaria': True,
    'categoria': 'trabalho'
})
```

**Dica:** Use operadores lógicos (AND) para combinar critérios.

In [20]:
def filtrar_emails_por_criterios(emails, criterios):
    """
    Filtra emails por múltiplos critérios
    
    Args:
        emails: Lista de emails processados
        criterios: Dicionário com critérios de filtro
    
    Returns:
        Lista de emails que atendem aos critérios
    """
    emails_filtrados = []
    
    for email in emails:
        atende_criterios = True
        
        # Filtro por categoria
        if 'categoria' in criterios:
            if email.categoria != criterios['categoria']:
                atende_criterios = False
        
        # Filtro por prioridade
        if 'prioridade' in criterios:
            if email.prioridade != criterios['prioridade']:
                atende_criterios = False
        
        # Filtro por urgência mínima
        if 'urgencia_minima' in criterios:
            if email.urgencia < criterios['urgencia_minima']:
                atende_criterios = False
        
        # Filtro por ação necessária
        if 'acao_necessaria' in criterios:
            if email.acao_necessaria != criterios['acao_necessaria']:
                atende_criterios = False
        
        # Filtro por palavras-chave
        if 'palavras_chave' in criterios:
            palavras_encontradas = any(
                palavra.lower() in ' '.join(email.palavras_chave).lower()
                for palavra in criterios['palavras_chave']
            )
            if not palavras_encontradas:
                atende_criterios = False
        
        if atende_criterios:
            emails_filtrados.append(email)
    
    return emails_filtrados


print("\n🔍 Testando filtros...")
emails_urgentes = filtrar_emails_por_criterios(resultados_ind, {
    'urgencia_minima': 4,
    'acao_necessaria': True
})
print(f"📧 Emails urgentes com ação: {len(emails_urgentes)}")

emails_trabalho = filtrar_emails_por_criterios(resultados_ind, {
    'categoria': 'trabalho'
})
print(f"📧 Emails de trabalho: {len(emails_trabalho)}")


🔍 Testando filtros...
📧 Emails urgentes com ação: 2
📧 Emails de trabalho: 1


### **Desafio 2: Análise de Sentimento** 😊
**Objetivo:** Implementar análise automática de sentimento dos emails para entender o tom da comunicação.

**O que você deve fazer:**
- Criar um prompt para classificar sentimento (POSITIVO/NEGATIVO/NEUTRO)
- Implementar análise baseada no resumo do email
- Criar função que processe múltiplos emails
- Gerar relatório de sentimento por email
- Integrar com o sistema de urgência para correlacionar sentimento e urgência

**Exemplo de saída:**

📧 "Reunião cancelada": NEGATIVO (Urgência: 5)

📧 "Convite para aniversário": POSITIVO (Urgência: 2)

📧 "Newsletter semanal": NEUTRO (Urgência: 1)


**Dica:** Use o resumo do email para análise, é mais eficiente que o texto completo.


In [21]:
def analisar_sentimento_emails(emails):
    """
    Analisa o sentimento dos emails
    
    Args:
        emails: Lista de emails processados
    
    Returns:
        Dicionário com análise de sentimento
    """
    from langchain.prompts import PromptTemplate
    from langchain_openai import ChatOpenAI
    from langchain_core.output_parsers import StrOutputParser
    
    # Configura análise de sentimento
    chat_model = ChatOpenAI(temperature=0.0, model_name="gpt-3.5-turbo")
    output_parser = StrOutputParser()
    
    prompt_sentimento = PromptTemplate(
        template="""
        Analise o sentimento do seguinte email e responda apenas com uma das opções:
        - POSITIVO: Email com tom positivo, amigável, otimista
        - NEGATIVO: Email com tom negativo, crítico, pessimista
        - NEUTRO: Email com tom neutro, informativo, formal
        
        Email:
        {email}
        
        Sentimento:
        """,
        input_variables=["email"]
    )
    
    chain_sentimento = prompt_sentimento | chat_model | output_parser
    
    analises = []
    for email in emails:
        try:
            sentimento = chain_sentimento.invoke({"email": email.resumo})
            analises.append({
                "assunto": email.assunto,
                "sentimento": sentimento.strip(),
                "urgencia": email.urgencia
            })
        except Exception as e:
            print(f"❌ Erro ao analisar sentimento: {e}")
            analises.append({
                "assunto": email.assunto,
                "sentimento": "NEUTRO",
                "urgencia": email.urgencia
            })
    
    return analises

print("\n😊 Testando análise de sentimento...")
sentimentos = analisar_sentimento_emails(resultados_ind)
for sentimento in sentimentos[:3]:  # Mostra apenas os primeiros 3
    print(f"   {sentimento['assunto']}: {sentimento['sentimento']}")



😊 Testando análise de sentimento...
   URGENTE: Reunião cancelada - Projeto X: NEGATIVO
   Convite para aniversário: POSITIVO
   GANHE R$ 10.000 POR MÊS TRABALHANDO DE CASA!: NEGATIVO


### **Desafio 3: Dashboard de Estatísticas** 📊
**Objetivo:** Criar um dashboard completo com métricas e estatísticas dos emails processados.

**O que você deve fazer:**
- Calcular total de emails processados
- Contar emails por categoria e prioridade
- Calcular estatísticas de urgência (média, máxima, mínima)
- Contar emails que requerem ação
- Identificar palavras-chave mais frequentes
- Calcular percentuais e proporções
- Criar estrutura de dados organizada para visualização

**Métricas obrigatórias:**
- Total de emails
- Distribuição por categoria
- Distribuição por prioridade
- Estatísticas de urgência
- Percentual de emails com ação necessária
- Top 10 palavras-chave mais frequentes

**Dica:** Use `Counter` do Python para contar frequências.


In [23]:
def criar_dashboard_emails(emails):
    """
    Cria dashboard com estatísticas dos emails
    
    Args:
        emails: Lista de emails processados
    
    Returns:
        Dicionário com estatísticas completas
    """
    if not emails:
        return {"erro": "Nenhum email para analisar"}
    
    # Estatísticas básicas
    total_emails = len(emails)
    
    # Contagem por categoria
    categorias = {}
    for email in emails:
        categoria = email.categoria
        categorias[categoria] = categorias.get(categoria, 0) + 1
    
    # Contagem por prioridade
    prioridades = {}
    for email in emails:
        prioridade = email.prioridade
        prioridades[prioridade] = prioridades.get(prioridade, 0) + 1
    
    # Estatísticas de urgência
    urgencias = [email.urgencia for email in emails]
    urgencia_media = sum(urgencias) / len(urgencias)
    urgencia_max = max(urgencias)
    urgencia_min = min(urgencias)
    
    # Emails que requerem ação
    emails_com_acao = sum(1 for email in emails if email.acao_necessaria)
    
    # Palavras-chave mais frequentes
    todas_palavras = []
    for email in emails:
        todas_palavras.extend(email.palavras_chave)
    
    from collections import Counter
    palavras_frequentes = Counter(todas_palavras).most_common(10)
    
    dashboard = {
        "total_emails": total_emails,
        "categorias": categorias,
        "prioridades": prioridades,
        "urgencia": {
            "media": round(urgencia_media, 2),
            "maxima": urgencia_max,
            "minima": urgencia_min
        },
        "emails_com_acao": emails_com_acao,
        "percentual_acao": round((emails_com_acao / total_emails) * 100, 2),
        "palavras_frequentes": palavras_frequentes
    }
    
    return dashboard


print("\n📊 Testando dashboard...")
dashboard = criar_dashboard_emails(resultados_ind)
print(f"📧 Total de emails: {dashboard['total_emails']}")
print(f"📊 Categorias: {dashboard['categorias']}")
print(f"⚡ Urgência média: {dashboard['urgencia']['media']}")
print(f"📧 Emails com ação: {dashboard['emails_com_acao']} ({dashboard['percentual_acao']}%)")



📊 Testando dashboard...
📧 Total de emails: 5
📊 Categorias: {'trabalho': 1, 'pessoal': 1, 'spam': 1, 'newsletter': 1, 'Financeiro': 1}
⚡ Urgência média: 3.6
📧 Emails com ação: 3 (60.0%)


### **Desafio 4: Exportação Avançada** 💾
**Objetivo:** Implementar sistema de exportação de dados em formato estruturado para análise posterior.

**O que você deve fazer:**
- Exportar dados em formato JSON estruturado
- Incluir metadados (timestamp, versão, total de emails)
- Exportar dashboard completo com estatísticas
- Exportar lista detalhada de todos os emails processados
- Criar função que salve arquivo com nome personalizado
- Incluir informações de validação e integridade dos dados

**Estrutura do arquivo JSON:**
```json
{
  "metadata": {
    "timestamp": "2024-03-20T10:30:00",
    "versao": "1.0",
    "total_emails": 5
  },
  "dashboard": {
    "categorias": {"trabalho": 2, "pessoal": 1},
    "urgencia": {"media": 3.2, "maxima": 5}
  },
  "emails": [...]
}
```

**Dica:** Use `json.dump()` com `ensure_ascii=False` para caracteres especiais.


In [24]:
def exportar_dados_completos(emails, dashboard, filename="relatorio_emails_completo.json"):
    """
    Exporta dados completos em formato JSON
    
    Args:
        emails: Lista de emails processados
        dashboard: Dashboard com estatísticas
        filename: Nome do arquivo para exportar
    """
    import json
    from datetime import datetime
    
    # Prepara dados para exportação
    dados_export = {
        "metadata": {
            "timestamp": datetime.now().isoformat(),
            "versao": "1.0",
            "total_emails": len(emails)
        },
        "dashboard": dashboard,
        "emails": []
    }
    
    # Adiciona emails processados
    for email in emails:
        dados_export["emails"].append({
            "remetente": email.remetente,
            "email_remetente": email.email_remetente,
            "assunto": email.assunto,
            "data_envio": email.data_envio,
            "categoria": email.categoria,
            "prioridade": email.prioridade,
            "urgencia": email.urgencia,
            "acao_necessaria": email.acao_necessaria,
            "palavras_chave": email.palavras_chave,
            "resumo": email.resumo
        })
    
    # Salva arquivo
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(dados_export, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Dados exportados para {filename}")
    return filename

print("\n💾 Testando exportação...")
arquivo_exportado = exportar_dados_completos(resultados_ind, dashboard)
print(f"✅ Arquivo exportado: {arquivo_exportado}")


💾 Testando exportação...
✅ Dados exportados para relatorio_emails_completo.json
✅ Arquivo exportado: relatorio_emails_completo.json
